In [70]:
#Importing libraries
import numpy as np
import pandas as pd
import warnings

In [71]:
warnings.filterwarnings('ignore')

In [49]:
def CommonInf(Introduction, ds):
    string = """
    {}
    
    Common information:
    Head:
    {}
    
    Null values:
    {}
    
    Row size:
    {}
        """.format(Introduction, ds.head(5), ds.isnull().sum(), ds.shape[0] )
    return string

In [42]:
def NanValues(ds):
    #Verify how many null rows there are in the ds doesnt matter the column
    NullValues = ds.isnull().sum().sum()
    
    #Validation about null values
    if(NullValues > 0):
        return ds.dropna()
    else:
        return ds

In [78]:
def dateFormat(ds):
    ds['DateSales']= pd.to_datetime(ds['DateSales'])
    
    ds['DateSales'] = ds['DateSales'].dt.strftime('%Y/%m/%d')
    return ds

In [97]:
def CategoricalRow(ds):
    ds['Model'] = pd.factorize(ds['Model'])[0] 
    return pd.DataFrame(ds)

In [107]:
def YearSale(ds):
    ds['DateSales']= pd.to_datetime(ds['DateSales'])
    ds['Year Sale'] = ds['DateSales'].dt.strftime('%Y')
    return ds

In [ ]:
def DoubletoInt(ds):
    ds['Year'] = ds

In [101]:
def ExtractAndTransform():
    ds = pd.read_csv('dataset.csv')
    #print(CommonInf('Initial dataset', ds))
    
    #Validation for nan values
    ds = NanValues(ds)
    #print(CommonInf('Dataset after null validation',ds))
    
    #Validation for standart date format yyyy/mm/dd
    ds = dateFormat(ds)
    
    #Year of sale
    ds = YearSale(ds)
    
    #Converting model to numeric values
    ds = CategoricalRow(ds)
    
    #Casting year column from double to int
    ds= DoubletoInt(ds)
    
    print(ds.head(5))
    

In [108]:
#Main function
if __name__ == "__main__":
    ExtractAndTransform()

   id         Car  Model                Vin    year  DateSales       Address  \
1   2  Volkswagen      0  19UUA66285A189929  2010.0 2013-09-10  PO Box 48548   
2   3        Ford      1  1G6YX36D365936025  1995.0 2006-10-22      Suite 36   
3   4        Audi      2  WBAPK53569A461317  2000.0 2005-02-18  PO Box 75995   
5   6   Chevrolet      3  1GKS1HE0XBR157468  1993.0 2011-07-28  PO Box 27981   
6   7  Mitsubishi      4  WAUKH78E67A197316  2000.0 2017-08-22      Suite 78   

  Year Sale  
1      2013  
2      2006  
3      2005  
5      2011  
6      2017  
